# Выделение ключевых слов

__Идея:__ я хочу проверить, насколько хорошо будет работать матрица совстречаемости, которая будет основана на полном проходе по всему тексту (а не случайному выбору, пусть и большое количество раз, как в random walk, который мы обсуждали на парах).

_Дополнительная идея:_ посмотреть, как результат будет зависеть от величины «окна», которую мы зададим.

## Алгоритм

### Обработка текста

In [1]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [2]:
mystem = Mystem()
stops = stopwords.words("russian")
punctuators = punctuation + "«–»— \r\n"

In [5]:
def lemmatize_text(text):
    """Из исходного текста делает список лемм, очищенных от пунктуации
    и стоп-слов.
    
    :arg text (str): исходный текст, как он загружен из файла
    :returns clean_text (list of str): список лемм текста в том порядке, в
    котором они были в тексте
    """
    lemmas_raw = [parse["analysis"][0]["lex"] for parse in mystem.analyze(text)
                  if parse.get("analysis")]
    clean_text = [lemma_raw for lemma_raw in lemmas_raw
                 if lemma_raw not in stops and lemma_raw not in punctuators]        
    return clean_text

In [11]:
clean_text = lemmatize_text(sample)

Список слов, где все слова повторяются только один раз, нужен для составления графа:

In [12]:
word_list = list(set(clean_text))

### Составление матрицы смежности

In [13]:
import numpy as np

In [27]:
def fill_co_occurence_matrix(text_lemmas, word_list, window):
    """Собирает матрицу совстречаемости из текста.
    
    :arg text_lemmas (list of str): лемматизированный и почищенный текст
    :arg word_list (list of str): список уникальных слов
    :arg window (int): окно, в котором мы считаем совстречаемость
    
    :returns M_co_occur (np.ndarray): матрица совстречаемости
    """
    M_co_occur = np.zeros((len(word_list), len(word_list)))
    for lemma_ind, lemma in enumerate(text_lemmas):
        lemma_matrix_index = word_list.index(lemma)
        ind_left = max(0, lemma_ind - window)
        ind_right = min(lemma_ind + window, len(text_lemmas))
        for ind_neighbor in range(ind_left, ind_right):
            word_neighbor = clean_text[ind_neighbor]
            neighbor_matrix_index = word_list.index(word_neighbor)
            M_co_occur[lemma_matrix_index][neighbor_matrix_index] += 1
    return M_co_occur

Предположим, что ключевыми будут те слова, которые соединяются с наибольшим количеством других слов => их совстречаемость будет самой высокой.

In [ ]:
def extract_keywords_from_matrix(matrix, word_list, threshold):
    """Из матрицы совстречаемости выбирает сколько-то слов с наибольшей 
    совстречаемостью.
    
    :arg matrix (np.ndarray): матрица совстречаемости
    :arg word_list (list of str): список уникальных слов, на котором строилась 
    матрица
    :arg threshold (int): сколько первых значений совстречаемости брать
    
    :returns keywords (list of str): сами ключевые слова
    """
    kw_indices = np.where(np.argmax(matrix, axis=1) <= threshold)[0]
    keywords = [word_list[kw_ind] for kw_ind in kw_indices]
    return keywords

### Измеряем качество

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
def evaluate_kws(kws_true, kws_predicted):
    precision = precision_score(kws_true, kws_predicted)
    recall = recall_score(kws_true, kws_predicted)
    f1 = f1_score(kws_true, kws_predicted)
    print("\tprecision={:.4f}, recall={:.4f}, F1={:.4f}".format(precision, recall, f1))
    return precision, recall, f1

### Всё вместе

In [ ]:
def pipeline_for_texts(text, kw_true):
    text_lemmas = lemmatize_text(text)
    word_list = list(set(clean_text))
    for window in range(1, 6):
        print("window={}".format(window))
        M = fill_co_occurence_matrix(text_lemmas, word_list, window)
        for thresh in range(1, 4):
            kw = extract_keywords_from_matrix(M, word_list, thresh)
            print("\ttop-{} co-occurences: {}".format(thresh, kw))
            precision, recall, f1 = evaluate_kws(kw_true, kw)